In [ ]:
!pip install -qq nest_asyncio pydantic pydantic_ai rich html2text python-dotenv

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from rich import print as pprint

In [ ]:
from pydantic_ai import Agent

test_agent = Agent(
    model="groq:llama-3.2-3b-preview",
    system_prompt="You are a commercial international Pilot."
)

response = test_agent.run_sync("Guide me and give me a path to become a commercial pilot like you")

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import html2text
import re

def clean_text(text):
    """Clean extracted text by removing extra whitespace and empty lines"""
    text = re.sub(r'\n\s*\n', '\n\n', text.strip())
    return text

def scrape_website(url, selector=None):
    """
    Scrapes data from a website and converts HTML to Markdown.
    
    Parameters:
    url (str): The URL of the website to scrape
    selector (str, optional): CSS selector to target specific elements
    
    Returns:
    str: Markdown formatted text
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove unwanted elements
        for element in soup.select('script, style, nav, footer, header'):
            element.decompose()
        
        # Convert to Markdown
        h = html2text.HTML2Text()
        h.ignore_links = False
        h.ignore_images = False
        h.body_width = 0  # No wrapping
        
        if selector:
            elements = soup.select(selector)
            content = '\n\n'.join(h.handle(str(element)) for element in elements)
        else:
            content = h.handle(str(soup.body))
        
        # Clean and save content
        content = clean_text(content)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        with open(f'scraped_content_{timestamp}.md', 'w', encoding='utf-8') as file:
            file.write(content)
            
        return content
        
    except Exception as e:
        print(f"Error: {e}")
        return ""

website_url = "https://www.cs.cmu.edu/~fgandon/miscellaneous/murphy/"
content = scrape_website(website_url)